In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"
%run "D3. Simulation Functions.ipynb"

baseball_path = r'C:\Users\james\Documents\MLB\Database'

Code was last run on: 2023-11-04


In [2]:
db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [3]:
todaysdate

'20231104'

### A01. DraftKings

In [ ]:
%run "A01. DraftKings.ipynb"

##### 1. Contests

In [ ]:
# Scrape contests
contest_df = contests(todaysdate)
# To csv
contest_df.to_csv(os.path.join(baseball_path, 'A01. DraftKings', '1. Contests', f'Contests {todaysdate}.csv'), index=False)
# To database
contest_df.to_sql(f'Contests {todaysdate}', con=engine, index=False, if_exists='replace')

In [ ]:
# Select contests of interest
subset_df = contest_df[(contest_df['Name'].str.contains('Four-Seamer')) & (~contest_df['Name'].str.contains('Turbo|vs'))].reset_index(drop=True)
print(subset_df.to_string())

##### 2. Draftables

In [ ]:
# Loop over contests of interest
for i in range(len(subset_df)):
    # Extract draftGroupId
    draftGroupId = subset_df['draftGroupId'][i]
    try:
        # Scrape draftables (DK Salaries)
        draftable_df = draftables(draftGroupId)
        # To csv
        draftable_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), index=False, encoding='iso-8859-1')
        # To database
        draftable_df.to_sql(f'Draftables {draftGroupId}', con=engine, index=False, if_exists='replace')
        print(f"Saved {subset_df['draftGroupId'][i]}")
    except:
        print(f"Didn't save {subset_df['draftGroupId'][i]}")

##### 3. Payouts

In [ ]:
# Loop over contests of interest
for i in range(len(subset_df)):
    # Extract contestKey
    contestKey = subset_df['contestKey'][i]
    # Scrape payouts
    payout_df = payouts(contestKey)
    # To csv
    payout_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "3. Payouts", f"Payouts {contestKey}.csv"), index=False, encoding='iso-8859-1')
    # To database
    payout_df.to_sql(f'Payouts {contestKey}', con=engine, index=False, if_exists='replace')

##### 4. Results

In [ ]:
# Read in yesterday's contest 
# yesterdays_contest_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '1. Contests', f'Contests {yesterdaysdate}.csv'))
sql_query = f'SELECT * FROM "Contests {yesterdaysdate}"'
yesterdays_contest_df = pd.read_sql_query(sql_query, con=engine)

# Select contests of interest
yesterdays_subset_df = yesterdays_contest_df[(yesterdays_contest_df['Name'].str.contains('Four-Seamer')) & (~yesterdays_contest_df['Name'].str.contains('Turbo|vs'))].reset_index(drop=True)
print(yesterdays_subset_df.to_string())

##### 5. Entry Results and 6. Player Results

In [ ]:
# Loop over yesterday's contests
for i in range(len(yesterdays_subset_df)):
    # Extract contestKey
    contestKey = yesterdays_subset_df['contestKey'][i]
    print(contestKey)
    # Scrape results
    results(contestKey)
    time.sleep(15)
    # Read into pandas
    results_df = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "4. Results", f"contest-standings-{contestKey}.csv"))
    
    # Entry results
    entry_results_df = entry_results(results_df)
    entry_results_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "5. Entry Results", f"Entry Results {contestKey}.csv"), index=False, encoding='iso-8859-1')
    entry_results_df.to_sql(f'Entry Results {contestKey}', con=engine, index=False, if_exists='replace')
    # Player results
    player_results_df = player_results(results_df)
    player_results_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results", f"Player Results {contestKey}.csv"), index=False, encoding='iso-8859-1')
    player_results_df.to_sql(f'Player Results {contestKey}', con=engine, index=False, if_exists='replace')

### A02. MLB API

In [4]:
%run "A02. MLB API.ipynb"

##### 1. Stats API 

In [5]:
statsapi_df = plays_statsapi("03/29/2023", "11/15/2023")
statsapi_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "1. Stats API", "Stats API 2023.csv"), index=False, encoding='iso-8859-1')
statsapi_df.astype('str').to_sql('Stats API 2023', con=engine, index=False, if_exists='replace')
del statsapi_df

##### 2. Statcast

In [6]:
statcast_df = plays_statcast("2023-03-15", "2023-11-15")
statcast_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "2. Statcast", "Statcast 2023.csv"), index=False, encoding='iso-8859-1')
statcast_df.astype('str').to_sql('Statcast 2023', con=engine, index=False, if_exists='replace')
del statcast_df

This is a large query, it may take a moment to complete


100%|██████████| 246/246 [03:25<00:00,  1.20it/s]


##### 3. Dataset

In [5]:
%%time
complete_dataset = create_inputs(start_year=2015, end_year=2023, short=50, long=300)
complete_dataset[model_inputs_plus].to_sql('Dataset', con=engine, index=False, if_exists='replace')
# del complete_dataset

CPU times: total: 10min 26s
Wall time: 26min 5s


1597990

### A03. Steamer

In [14]:
%run "A03. Steamer.ipynb"

In [15]:
# Download Steamer weekly logs and save them to csv
download_logs()

<IPython.core.display.Javascript object>

Moved 'C:\Users\james\Downloads\steamer_pitchers_weekly_log.csv' to 'C:\Users\james\Documents\MLB\Database\A03. Steamer'.
Moved 'C:\Users\james\Downloads\steamer_hitters_weekly_log.csv' to 'C:\Users\james\Documents\MLB\Database\A03. Steamer'.


In [16]:
# Read in Steamer hitters 
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
# To database
steamer_hitters_df.to_sql('Steamer Hitters', con=engine, index=False, if_exists='replace')

# Read in Steamer pitchers
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
# To database
steamer_pitchers_df.to_sql('Steamer Pitchers', con=engine, index=False, if_exists='replace')

1322537

# Below this point needs to be run prior to simulating, may be best moved elsewhere (Morning, Evening files)

### A04. Bullpens

In [ ]:
%run "A04. Bullpens.ipynb"

In [ ]:
# Make it so you can do the past like with rosters

# Create path
if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {todaysdate}")):
    os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {todaysdate}"))
    time.sleep(1)
else:
    print("Already exists")    
    
# Scrape bullpens
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))
for i in range(len(team_map)):
    # Extract team's website URL
    mlburl = team_map['MLBURL'][i]
    # Extract team's Baseball Reference abbreviation
    bbrefteam = team_map['BBREFTEAM'][i]
    # Scrape bullpens
    bullpen_df = scrape_bullpen(mlburl, header, bbrefteam, date=todaysdate)
    # To csv
    bullpen_df.to_csv(os.path.join(baseball_path, "A04. Bullpens", f"Bullpens {todaysdate}", f"Bullpen {bbrefteam} {todaysdate}.csv"), index=False, encoding='iso-8859-1')
    # To database
    bullpen_df.to_sql(f'Bullpen {bbrefteam} {todaysdate}', con=engine, index=False, if_exists='replace')

### A05. Rosters

In [16]:
%run "A05. Rosters.ipynb"

##### 1. Batting Orders

In [ ]:
# Read in today's games
games = statsapi.schedule(start_date=todaysdate_slash, end_date=todaysdate_slash)

# Create path
if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\1. Batting Orders', f"Batting Orders {todaysdate}")):
    os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\1. Batting Orders', f"Batting Orders {todaysdate}"))
    time.sleep(1)
else:
    print("Already exists")    

# Scrape batting orders
# Loop over games
for game in games:
    # Extract game ID
    game_id = game['game_id']
    # Extract team IDs
    away_id = game['away_id']
    home_id = game['home_id']
    
    # Scrape away team batting order
    away_order_df = order(game_id, away_id, todaysdate, 'away')
    away_team = team_map.loc[team_map['teamId'] == away_id, 'BBREFTEAM'].values[0]
    # To csv
    away_order_df.to_csv(os.path.join(baseball_path, "A05. Rosters", "1. Batting Orders", f"Batting Orders {todaysdate}", f"Batting Order {away_team} {game_id}.csv"), index=False, encoding='iso-8859-1')
    # To database
    away_order_df.to_sql(f'Batting Order {away_team} {game_id}', con=engine, index=False, if_exists='replace')
    
    # Scrape home team batting order
    home_order_df = order(game_id, home_id, todaysdate, 'home')
    home_team = team_map.loc[team_map['teamId'] == home_id, 'BBREFTEAM'].values[0]
    # To csv
    home_order_df.to_csv(os.path.join(baseball_path, "A05. Rosters", "1. Batting Orders", f"Batting Orders {todaysdate}", f"Batting Order {home_team} {game_id}.csv"), index=False, encoding='iso-8859-1')
    # To database
    home_order_df.to_sql(f'Batting Order {home_team} {game_id}', con=engine, index=False, if_exists='replace')

##### 2. Rosters

In [30]:
# Scrape rosters
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# from datetime import datetime, timedelta

# Set dates
start_date_str = "20230928"
end_date_str = "20231101"

# Convert to datetime 
start_date = datetime.strptime(start_date_str, "%Y%m%d")
end_date = datetime.strptime(end_date_str, "%Y%m%d")

# Start with start date
current_date = start_date

# Loop over dates
while current_date <= end_date:
    # Create date string
    formatted_date_str = current_date.strftime("%Y%m%d")
    print(formatted_date_str)
    current_date += timedelta(days=1)
    
    # Create path (if it doesn't exist)
    if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\2. Rosters', f"Rosters {formatted_date_str}")):
        os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\2. Rosters', f"Rosters {formatted_date_str}"))
        time.sleep(1)
    else:
        print("Already exists")    

    
    # Loop over teams
    for i in range(len(team_map)):
        # Extract teamId
        teamId = team_map['teamId'][i]
        # Extrct team's Baseball Reference abbreviation
        bbrefteam = team_map['BBREFTEAM'][i]
        # Scrape rosters
        roster_df = roster(teamId, formatted_date_str)
        # To csv
        roster_df.to_csv(os.path.join(baseball_path, "A05. Rosters", "2. Rosters", f"Rosters {formatted_date_str}", f"Roster {bbrefteam} {formatted_date_str}.csv"), index=False, encoding='iso-8859-1')
        # To database
        roster_df.to_sql(f'Roster {bbrefteam} {formatted_date_str}', con=engine, index=False, if_exists='replace')

20230928
Already exists
20230929
Already exists
20230930
Already exists
20231001
Already exists
20231002
Already exists
20231003
Already exists
20231004
20231005
Already exists
20231006
20231007
Already exists
20231008
Already exists
20231009
Already exists
20231010
Already exists
20231011
Already exists
20231012
20231013
Already exists
20231014
Already exists
20231015
20231016
Already exists
20231017
Already exists
20231018
20231019
Already exists
20231020
Already exists
20231021
20231022
Already exists
20231023
20231024
20231025
20231026
20231027
20231028
20231029
20231030
Already exists
20231031
20231101
Already exists


### A06. Weather

In [ ]:
%run "A06. Weather.ipynb"

##### 1. Swish Analytics

In [ ]:
# Scrape Swish Analytics
swishanalytics_df = swishanalytics(todaysdate)
# To csv
swishanalytics_df.to_csv(os.path.join(baseball_path, "A06. Weather", "1. Swish Analytics", f"Swish Analytics {todaysdate}.csv"), index=False)
# To database
swishanalytics_df.to_sql(f'Swish Analytics {todaysdate}', con=engine, index=False, if_exists='replace')

##### 2. RotoGrinders

In [ ]:
# Scrape RotoGrinders
rotogrinders_df = rotogrinders(todaysdate)
# To csv
rotogrinders_df.to_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"), index=False)
# To database
rotogrinders_df.to_sql(f'RotoGrinders {todaysdate}', con=engine, index=False, if_exists='replace')

##### 3. Ballpark Pal

In [ ]:
# Scrape Ballpark Pal
ballparkpal_df = ballparkpal(todaysdate)
# To csv
ballparkpal_df.to_csv(os.path.join(baseball_path, "A06. Weather", "3. Ballpark Pal", f"Ballpark Pal {todaysdate}.csv"), index=False)
# To database
ballparkpal_df.to_sql(f'Ballpark Pal {todaysdate}', con=engine, index=False, if_exists='replace')

### A07. Projections

In [ ]:
%run "A07. Projections.ipynb"

##### 1. DFF

In [ ]:
# Scrape slates
dff_slates_df = dff_slates(todaysdate)
# To csv
dff_slates_df.to_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "1. Slates", f"DFF Slates {todaysdate}.csv"), index=False)
# To database
dff_slates_df.to_sql(f'DFF Slates {todaysdate}', con=engine, index=False, if_exists='replace')

In [ ]:
# Loop over slates
for code in dff_slates_df['URL']:
    # Scrape projections
    dff_projections_df = dff_projections(todaysdate, code)
    # To csv
    dff_projections_df.to_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "2. Projections", f"DFF Projections {code}.csv"), index=False)
    # To database
    dff_projections_df.to_sql(f'DFF Projections {code}', con=engine, index=False, if_exists='replace')

##### 2. RotoWire

In [ ]:
# Scrape slates
roto_slates_df = roto_slates(todaysdate)
# To csv
roto_slates_df.to_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "1. Slates", f"RotoWire Slates {todaysdate}.csv"), index=False)
# To database
roto_slates_df.to_sql(f'RotoWire Slates {todaysdate}', con=engine, index=False, if_exists='replace')

In [ ]:
# Loop over slates
for code in roto_slates_df['slateID']:
    # Scrape projections
    roto_projections_df = roto_projections(todaysdate, code)
    # To csv
    roto_projections_df.to_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "2. Projections", f"RotoWire Projections {code}.csv"), index=False)
    # To database
    roto_projections_df.to_sql(f'RotoWire Projections {code}', con=engine, index=False, if_exists='replace')    

### A08. Odds

In [ ]:
%run "A08. Odds.ipynb"

In [ ]:
# Scrape odds
odds_df = odds(todaysdate)
# To csv
odds_df.to_csv(os.path.join(baseball_path, "A08. Odds", f"Odds {todaysdate}.csv"), index=False)
# To database
odds_df.to_sql(f'Odds {todaysdate}', con=engine, index=False, if_exists='replace')    

In [ ]:
# One time

In [ ]:
# # Read in park factors
# park_factors_l = pd.read_excel(r"C:\Users\james\Documents\MLB\Database\Utilities\Statcast Park Factors.xlsx", sheet_name='L')
# park_factors_l['batSide'] = "L"
# park_factors_r = pd.read_excel(r"C:\Users\james\Documents\MLB\Database\Utilities\Statcast Park Factors.xlsx", sheet_name='R')
# park_factors_r['batSide'] = "R"

# park_factors = pd.concat([park_factors_l, park_factors_r], axis=0)

# park_factors.to_sql(f'Statcast Park Factors', con=engine, index=False, if_exists='replace')    

In [ ]:
# # Team map
# team_map = pd.read_csv(r"C:\Users\james\Documents\MLB\Database\Utilities\Team Map.csv")
# team_map.to_sql('Team Map', con=engine, index=False, if_exists='replace')